In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

In [ ]:
# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

# Import hvplot
import hvplot.pandas

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', size='Humidity', alpha=0.7, color='blue', hover_cols=['City', 'Humidity'],
                                       xlim=(-180, 180), ylim=(-90, 90), title="City Humidity", 
                                       xlabel="Longitude", ylabel="Latitude", frame_width=800, frame_height=500)

# Display the map
map_plot


In [ ]:
# Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

# Define the criteria for your ideal weather conditions
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Wind Speed"] < 4.5)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


In [ ]:
# Step 3: Create a new DataFrame called hotel_df.

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


In [ ]:
# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

# Set parameters to search for a hotel
radius = 10000  # Set the radius to 10,000 meters
params = {
    "type": "accommodation", 
    "limit": 1, 
}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    params["filter"] = {"around": f"{latitude},{longitude},{radius}"}
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set the base URL
    base_url = "https://api.geoapify.com/v2/places"

    response = requests.get(base_url, params=params).json()

    try:
        hotel_name = response["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


In [ ]:
# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

# Import hvplot
import hvplot.pandas

# Configure the map plot
map_plot = hotel_df.hvplot.scatter(x='Lng', y='Lat', size='Humidity', alpha=0.7, color='blue', hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'],
                                   xlim=(-180, 180), ylim=(-90, 90), title="City Humidity with Nearest Hotel Information",
                                   xlabel="Longitude", ylabel="Latitude", frame_width=800, frame_height=500)

# Display the map
map_plot
